In [13]:
#Library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
import math
import hdbscan


#Clustering
from sklearn.cluster import AgglomerativeClustering, KMeans
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from scipy.spatial.distance import pdist
from scipy import stats
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import pdist, squareform
from sklearn.preprocessing import StandardScaler
from scipy.cluster import hierarchy
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score


#Figure
import seaborn as sns

In [14]:
data_original = pd.read_csv("ustech.csv")
df = data_original
df


,dates,AAPL,MSFT,AMZN,GOOG,FB,TSLA,GOOGL,NVDA,PYPL,...,NFLX,INTC,CSCO,PEP,AVGO,TMUS,TXN,COST,QCOM,AMGN
0,01/01/2019,39.435,101.57,1501.97,1035.61,131.09,66.560,1044.96,133.50,84.09,...,267.66,46.93,43.33,110.48,254.28,63.4924,94.50,203.71,56.91,194.67
1,02/01/2019,39.480,101.12,1539.13,1045.85,135.68,62.024,1054.68,136.22,85.75,...,267.66,47.08,42.95,109.28,253.51,65.1393,94.45,204.76,57.40,191.90
2,03/01/2019,35.548,97.40,1500.28,1016.06,131.74,60.072,1025.47,127.99,82.09,...,271.20,44.49,41.07,108.26,230.96,64.8998,88.88,200.42,55.70,188.98
3,04/01/2019,37.065,101.93,1575.39,1070.71,137.95,63.538,1078.07,136.19,86.27,...,297.57,47.22,42.92,110.48,233.23,67.3652,92.90,206.24,56.60,195.44
4,05/01/2019,37.065,101.93,1575.39,1070.71,137.95,63.538,1078.07,136.19,86.27,...,297.57,47.22,42.92,110.48,233.23,67.3652,92.90,206.24,56.60,195.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
895,14/06/2021,130.480,259.89,3383.87,2527.04,336.77,617.690,2448.91,720.75,274.93,...,499.89,58.19,54.17,148.57,475.83,145.2600,190.26,383.76,137.31,241.19
896,15/06/2021,129.640,258.36,3383.13,2520.66,336.75,599.360,2428.39,711.54,271.89,...,491.90,57.99,53.79,148.58,470.74,145.6800,188.61,383.91,135.58,239.85
897,16/06/2021,130.150,257.38,3415.25,2513.93,331.08,604.870,2415.45,712.41,268.82,...,492.41,57.22,53.47,147.10,465.75,144.9100,187.73,379.41,134.95,239.58
898,17/06/2021,131.790,260.90,3489.24,2527.42,336.51,616.600,2434.87,746.29,278.11,...,498.34,57.18,53.13,148.52,471.17,146.4500,187.65,384.75,135.35,240.76


In [15]:
data_original = pd.read_csv("ustech.csv")
df = data_original
#Date columns become time series
df['dates'] = pd.to_datetime(df['dates'], dayfirst= True)
df.set_index('dates', inplace=True)

#Create new columns that show day-of-week based on the given dates
df['day_of_week'] = df.index.day_name()

#Remove the weekends since in those day the stock market did not work
df = df[~df['day_of_week'].isin([ "Saturday", "Sunday"])]



In [24]:
df[df.index == "2019-04-22"]

,AAPL,MSFT,AMZN,GOOG,FB,TSLA,GOOGL,NVDA,PYPL,ADBE,...,INTC,CSCO,PEP,AVGO,TMUS,TXN,COST,QCOM,AMGN,day_of_week
dates,,,,,,,,,,,,,,,,,,,,,
2019-04-22,51.133,123.76,1887.31,1248.84,181.44,52.55,1253.76,188.47,106.81,272.21,...,58.82,56.34,126.44,315.0,73.5837,114.95,243.42,81.97,176.36,Monday


In [569]:
#Remove the day-of-week columns since the weekend detection is done
df = df.iloc[:,0:(len(df.columns)-1)]
df